# Task 1: PageRank Stability on Evolving Graphs
## Graph evolution and PageRank values comparison
### Joris & Abdel

### Imports and general set-up

In [1]:
import networkx as nx
import plotly.plotly as py
import plotly.figure_factory as ff
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

Load edge list and create a graph

In [2]:
fh = open("canvas/hamster.edgelist", 'rb')
G = nx.read_edgelist(fh, create_using=nx.DiGraph())
fh.close()

In [3]:
np.random.seed(98)
random.seed(99)

In [4]:
def calc_pagerank(G_in, alpha = 0.85):
    return nx.pagerank(G_in, alpha=0.85)

In [5]:
pr_origin = calc_pagerank(G)

In [6]:
def create_dataframe(pr, G_in):
    df_edge_in = pd.DataFrame(list(G_in.in_degree()), columns=['node', 'in edges'])
    df_edge_out = pd.DataFrame(list(G_in.out_degree()), columns=['node', 'out edges'])
    df_rank = pd.DataFrame(list(pr.items()), columns=['node', 'score']).sort_values(by=['score'], ascending=False)
    df_temp = pd.merge(df_rank, df_edge_in, on='node')
    df_total = pd.merge(df_temp, df_edge_out, on='node')
    df_total.index = df_total.index + 1
    df_total.columns.name = 'rank'
    return df_total

In [70]:
df_origin = create_dataframe(pr_origin, G)
df_origin.head(10)

rank,node,score,in edges,out edges
1,404,0.042793,10,0
2,195,0.019961,80,1
3,77,0.018628,121,2
4,728,0.015530,10,0
5,36,0.011117,168,5
6,135,0.009544,49,8
7,192,0.009365,57,3
8,281,0.009304,32,0
9,136,0.008853,85,6
10,184,0.008296,80,3


It looks like node 404 is the best ranked page, following by 195 and 77. This means that these pages should be shown at the top by search engines.

### <font color="red">To do: Update the PageRank value calculation and analysis in this notebook with the extended/corrected/completed <i>(this still needs to be done)</i> version of the notebook of 1a</font>

## 1b. Graph Evolution and Pagerank values comparison

In this section the effects of graph evolutions are going to be studied in relation to an evaluation of the stability of PageRank. In particular, various methodologies are going to be devised and exploited in which graphical represesentations of a social network are going to be altered by the removal and/or addition of nodes and edges in these graphs. The original graph $G$, represents a social network of friendships and familylinks between users of the website <a>hamsterster.com</a>. Various functions which make it possible to change this graph are going to be given and explained. Some of these functions focus on the addition or removal of edges, while other focus on nodes. Some of these functions are going to do make choices at random, while others are also going to exploit randomness, but proporotional to the node degree and other statistics. The choice is made to analyze the effects of the functions which evolve the graphs on the original graph $G$. So the evaluation of the various functions which add/remove graphs is going to be done starting from the full and original graph $G$ for each of the given functions. 

<i>Note: A social network would naturally be described with an undirected graph. The social network data is, however, treated as a combination of target and source id's which faciliate the usage of this data as a directed graph for the sake of implementing and testing graph evolutions methods to evaluate the stability of PageRank. No implications or conclusions should be directly related to the actual structure of the social networks of the website</i>

### A. Removing and adding edges uniformly at random

For $n$ number of nodes do the following:
* select 1 node uniformly at random
* add or remove an incoming/outgoing at random 

In [100]:
#add/remove edges for all the nodes uniformly at random
def random_edges_uniform_random(G_in, number_of_nodes = 1):
    nr_of_edges_added = 0
    nr_of_edges_removed = 0
    
    list_of_nodes = list(G_in) # all the nodes
    # select uniformly at random nodes of which we are going to add/remove edges
    selected_nodes = list(np.random.choice(list_of_nodes, size = number_of_nodes, replace = False)) # default probability p is an uniform distribution
    
    for node in selected_nodes: 
        successors = list(G_in.successors(str(node))) # find the successors of this nodes
        predecessors = list(G_in.predecessors(str(node))) # find the predecessors of this node
        #find candidates for new edges
        unconnected_to = [n for n in list(G_in.nodes()) if not n in successors] # no outgoing edge to these nodes
        unconnected_from = [n for n in list(G_in.nodes()) if not n in predecessors] # no incoming edge from these nodes
        
        #add = bool(random.getrandbits(1)) # randomly add or remove an edge of this node
        add = False
        incoming =  bool(random.getrandbits(1)) # randomly add an outgoing/incoming edge
        if(add): # add an incoming/outgoing edge to node
            if(incoming): # add incoming edge
                if len(unconnected_from): #only add when unconnected_from is not empty
                    new = random.choice(unconnected_from)
                    G_in.add_edge(new, node)
                    print("\tnew edge:\t {} --> {}".format(new, node))
                    unconnected_from.remove(new)
                    predecessors.append(new)
            else: # add outgoing edge:
                if len(unconnected_to): #only add when unconnected_to is not empty
                    new = random.choice(unconnected_to)
                    G_in.add_edge(node, new)
                    print("\tnew edge:\t {} --> {}".format(node, new))
                    unconnected_to.remove(new)    
                    successors.append(new)
            nr_of_edges_added += 1
        else: # remove
            if(incoming): # remove incoming edge
                if len(predecessors): #only remove when predecessors is not empty
                    new = random.choice(predecessors)
                    G_in.remove_edge(new, node)
                    print("\tremove edge:\t {} --> {}".format(new, node))
                    predecessors.remove(new)
                    unconnected_from.append(new)
            else: # remove outgoing edge:
                if len(successors): #only remove when successors is not empty
                    new = random.choice(successors)
                    G_in.remove_edge(node, new)
                    print("\tremove edge:\t {} --> {}".format(node, new))
                    successors.remove(new)    
                    unconnected_to.append(new)
            nr_of_edges_removed += 1
            
    print("number of edges added: " + str(nr_of_edges_added))
    print("number of edges removed " + str(nr_of_edges_removed))
    
    return G_in

`random_edges_uniform_random` selects one node uniformly at random, and then for that specific node it uniformly at random adds or removes one outgoing or ingoing edge, which is also determined uniformly at random. Because the choice is made to analyze the effects of the functions which evolve the graphs on the original graph $G$, we call the function `random_edges_uniform_random` parameterized with a copy of $G$ and `number_of_nodes`$ = 100$. In other words, `random_edges_uniform_random` will either add or remove either an incoming or outgoing edge for each node of `number_of_nodes`. The result graph is stored in `G_random_edges_uniform_random`.

In [101]:
G_random_edges_uniform_random = random_edges_uniform_random(G.copy(), 400)

	remove edge:	 1088 --> 1089
	remove edge:	 1614 --> 57
	remove edge:	 1894 --> 1895
	remove edge:	 1215 --> 1216
	remove edge:	 1727 --> 1728
	remove edge:	 842 --> 11
	remove edge:	 758 --> 797
	remove edge:	 2024 --> 567
	remove edge:	 150 --> 885
	remove edge:	 951 --> 950
	remove edge:	 408 --> 347
	remove edge:	 182 --> 58
	remove edge:	 2412 --> 2415
	remove edge:	 790 --> 935
	remove edge:	 348 --> 773
	remove edge:	 2289 --> 732
	remove edge:	 1077 --> 1078
	remove edge:	 1101 --> 1102
	remove edge:	 1242 --> 1243
	remove edge:	 687 --> 514
	remove edge:	 847 --> 2
	remove edge:	 2063 --> 586
	remove edge:	 986 --> 987
	remove edge:	 2204 --> 2205
	remove edge:	 429 --> 336
	remove edge:	 2013 --> 21
	remove edge:	 474 --> 120
	remove edge:	 698 --> 410
	remove edge:	 59 --> 181
	remove edge:	 41 --> 948
	remove edge:	 2026 --> 2027
	remove edge:	 1488 --> 1489
	remove edge:	 1246 --> 1248
	remove edge:	 993 --> 994
	remove edge:	 2405 --> 2406
	remove edge:	 2046 --> 34
	remo

Next, we run `calc_pagerank` to calculate the new pagerank scores of `G_random_edges_uniform_random`. Thereafter a dataframe is created of the pagerank scores, together with for each node the number of incoming edges and outcoming edges. The nodes are sorted on the pagerank score, in descending order.

In [102]:
pr_random_edges_uniform_random = calc_pagerank(G_random_edges_uniform_random)
df_random_edges_uniform_random = create_dataframe(pr_random_edges_uniform_random, G_random_edges_uniform_random)
df_random_edges_uniform_random.head(10)

rank,node,score,in edges,out edges
1,404,0.043624,10,0
2,195,0.020481,80,1
3,77,0.019131,118,2
4,728,0.014947,10,0
5,36,0.011445,167,5
6,192,0.009451,57,3
7,135,0.009164,49,8
8,281,0.008927,30,0
9,184,0.008727,79,3
10,136,0.008476,85,6


Before the results are analyzed, lets first talk about some intuition what could happen when edges are added or removed uniformly at random. The original graph $G$ is a scale-free network, i.e. its degree distribution follows a power law, at least asymptiotically. This means that in this type of network structure, there will be many nodes with very low level of connectivity. And very few or except one node with exceptionally high degree of connectivity. So the nodes are very unequal in terms of how connected and influential the different nodes in the network are. Scale free networks describes a power or exponential relationship between the degree of connectivity a node has and the frequency of its occurence. This results in a highly centralized network. In the social network that is loaded from hamsterster.com, we have some people who have very many links into them, but there are also many people that have very few links into them. The power law distribution is often explained with reference to preferential attachment. Preferential attachment describes how a new node is linked amongst a number of nodes according to how much they already have. So those who already have a lot of links will receive more than those who have litte: the so called "rich get richer model". In the paragraphs D-E-F-G, preferential attachment proporional to some statistical measures is elaborated and analyzed. Paragraphs A-B-C will analyze the effects of adding/removing nodes/adges uniformly at random. In paragraph A, we add or remove only edges uniformly at random. In paragraph B nodes and corresponding edges are added uniformly at random. In paragraph C nodes and corresponding edges are removed uniformly at random. All modifications that will take place in paragraph A-B-C are determined uniformly at random, i.e. all nodes have equal probability to be chosen or to be removed. Scale free networks can be very robust or very fragile, depending on how we remove nodes (randomly or strategically). If we remove nodes uniformly at random, the network will be very robust to failure. This is because the vast majority of nodes have a very low degree of connectivity. Therefore, it is very likely that we will modify one of these insignificant nodes with little effect on the overall network.
So real word networks, like the network from hamsterster.com, are resilient to random attacks.  

When we add or remove only edges, we expect a constant average degree, i.e. the number of edges grows linearly with the number of nodes. Also, we expect that as the network grows, the distances between nodes grow. From the output of the function above, we see that $52$ edges are added, and $48$ edges are removed. When we look at the top ten nodes, we see that the top ten of `G_random_edges_uniform_random` exactly matches the top ten of the original graph $G$. Let's dive deeper into both graphs to look if there changed else.

First, we compare the $density$ between the original graph G and the graph `G_random_edges_uniform_random`. The density for directed graphs is: $d = \frac{E}{V(V-1)}$, where $E$ denotes the total number of edges and $V$ denotes the total number of nodes in the particular graph. We see that the top-ten remains the same, but that the density of the graphs increases when we add edges and decreases when we remove nodes. This totally makes sense of course.

In [96]:
nx.info(G_random_edges_uniform_random)

'Name: \nType: DiGraph\nNumber of nodes: 2426\nNumber of edges: 17031\nAverage in degree:   7.0202\nAverage out degree:   7.0202'

In [97]:
nx.info(G)

'Name: \nType: DiGraph\nNumber of nodes: 2426\nNumber of edges: 16631\nAverage in degree:   6.8553\nAverage out degree:   6.8553'

In [103]:
nx.density(G_random_edges_uniform_random)

0.0027705016955490774

In [104]:
nx.density(G)

0.002826935008201528

<b><font color = "red">todo: different dataframes. 1) only edges added 2)only edges removed 3) both addition and remove of edges</font></b>

When we determine inside the function whether we add or remove an edge, the top ten of nodes are equal to the top ten nodes. This makes totally sense. The purpose of the function is adding or removing edges uniformaly at random to `number_of_nodes` nodes. These selected nodes all have the same probability to be selected. As explained earlier, there are a lot of nodes in the network that have a low connectivity and only a few nodes with a very high connectivity. It is very likely that we will modify one of these insignificant nodes with little effect on the overall network and therefore we can explain that the top ten ranked nodes are not changed after adding or removing only nodes. 

### B. Adding nodes uniformly at random (copying model)

For $n$ iterations do the following:
* Make a new node instance $n$
* with a uniform random distribution pick $k$ nodes in the original graph
* copy the incoming/outgoing edges of the $k$ nodes for $n$
* choose with an unifrom distribution another node $l$ and add its edges also to $n$

<i>The last step might seem redundant at the moment, but later when the the $k$ nodes are going to be chosen with at random but proportional to a certain statistic, it makes sense to have a step in which you pick another node $l$ that is chosen with the opposite property so that the generation/stability of communities is ensured (i.e. power-law degree). In this implementation. This step is omitted, but it will be thus added in the functions that take statistical measures into consideration when choosing nodes at random</i>

In [117]:
#randomly add and remove nodes
#Edge Copying Model (slide 53 of Week6-SNA-Props)
def random_add_nodes_uniform(G_in, number_of_nodes = 1, k = 5):
    print("number of edges before :"+ str(len(G_in.edges())))
    for _ in range(number_of_nodes):
        #k is number of edges to be added, random integer 1 between 5
        k = random.randint(1, k) #select k random vertices
        #print("k = " + str(k))
        new_node = nx.number_of_nodes(G_in) + 1 #add node to graph
        # print("new node = " + str(new_node))
        
        list_of_nodes = list(G_in)  #create list of nodes
        
        G_in.add_node(str(new_node))   
        k_random_selected_nodes = np.random.choice(list_of_nodes, size = k, replace = False) # k nodes with a uniform distribution
        
        for node in k_random_selected_nodes:
            #print("node in k_random_selected_nodes = " + str(node))
            successors = list(G_in.successors(str(node)))
            #print("succesors are " + str(successors))
            for node_to in successors:
                G_in.add_edge(new_node, node_to) # add outgoing edges
            predecessors = list(G_in.predecessors(str(node)))
            for node_from in predecessors:
                G_in.add_edge(node_from, new_node) # add incoming edges
    print("number of edges after :"+str(len(G_in.edges())))
    return G_in

In [127]:
number_of_nodes_to_add_random = random.randint(1,  int(0.1 * len(list(G.copy()))))
print("number of nodes added: " + str(number_of_nodes_to_add_random))
G_random_add_nodes_uniform = random_add_nodes_uniform(G.copy(), number_of_nodes_to_add_random)

number of nodes added: 235
number of edges before :16631
number of edges after :19757


In [128]:
pr_random_add_nodes_uniform = calc_pagerank(G_random_add_nodes_uniform)
df_random_add_nodes_uniform = create_dataframe(pr_random_add_nodes_uniform, G_random_add_nodes_uniform)
df_random_add_nodes_uniform.head(10)

rank,node,score,in edges,out edges
1,404,0.042172,10,0
2,195,0.018128,84,1
3,77,0.017297,131,2
4,728,0.014063,10,0
5,36,0.010287,175,5
6,281,0.009745,36,0
7,126,0.009467,65,1
8,135,0.009011,56,8
9,192,0.008351,58,3
10,136,0.008253,94,6


`random_add_nodes_uniform` add nodes uniformly at random, while the degree distribution of the network still satisfies the power law distribution. We call the function, parameterized with `number_of_nodes` =  maximal 10% of the total number of nodes that are in $G$. What we see is that rank of some nodes in the top ten is changed. We even see a new node in top ten, node 126. This means that randomness can create new strong nodes, by adding new nodes and create new strong communities in the graph. However, since this all happens unformly at random, we can't guarantee that this happens every single run (which we can guarantee when we add proportional to some statistical measure, see D-E-F-G). The added nodes are linked, one by one, to $k$ random selected nodes. Note that every node in $G$ does have the same probability, so again, it is most likely that we select the insignificant nodes as nodes where we link the new nodes to. We see that node 126 is in the top then , but this is because node 184 has a lower pagerank score compared to the original graph $G$. This could be that the some of the new added nodes are linked to node 184, and because this nodes have a low pagerank score, it will have a negative effect on node 184. We can also look at the density of graph  `G_random_add_nodes_uniform`. Because we add nodes, the density should be lower compared to the original graph $G$. 

In [129]:
nx.density(G_random_add_nodes_uniform)

0.002392757080596565

In [130]:
nx.density(G)

0.002826935008201528

In [132]:
nx.info(G)

'Name: \nType: DiGraph\nNumber of nodes: 2426\nNumber of edges: 16631\nAverage in degree:   6.8553\nAverage out degree:   6.8553'

The output indeed confirms our hypotheses. Another aspect that we can look at is whether the number of edges grows linearly with the number of nodes. The number of edges before are 16631 and after the addition of 235 nodes we have 18104 edges. 1473 nodes are therefore added to the graph `G_random_add_nodes_uniform`. So the number of nodes has increased with a factor of approximately $1.1$. However, the number of edges is increased with $\frac{18104}{16631} = 1.08$. So the number of edges does linearly increase with number of nodes.  (<font color = "red">to-do: shouldn't number of edges not be more than 1.1?</font>)

<font color = "red">todo: analyze diamater (distance between nodes)</font>

### C. Removal of nodes uniformly at random

Let $n$ represent the number of nodes that should be removed. If the $number\_of\_nodes$ parameter is givem then $n = number\_of\_nodes$, if this parameter is not specfied by the caller we have $n = \lfloor(0.1 * total\_number\_of\_nodes(G\_in))\rfloor)$

For $n$ iterations do the following:
* Select a node $m$ uniformly at random (iterations are abstracted by $np.random.choice$)
* Remove this node and its respective edges from the graph

In [14]:
def random_removal_nodes_uniform(G_in, number_given = False, number_of_nodes = None):
    if (number_given & number_of_nodes < len(list(G_in))): # check if we do not remove too much nodes
        n = number_of_nodes
    else:
        n = int(0.1 * len(list(G_in)))  # max 10% of nodes
    #remove nodes and corresponding edges
    print("number of nodes before :"+ str(len(list(G_in))))
    list_of_nodes = list(G_in)
    selected_nodes = np.random.choice(list_of_nodes, size = n, replace = False)
    for m_remove in selected_nodes:
        G_in.remove_node(m_remove)
    print("number of nodes after :"+ str(len(list(G_in))))
    return G_in

In [15]:
G_random_removal_nodes_uniform = random_removal_nodes_uniform(G.copy(), True, 100)

number of nodes before :2426
number of nodes after :2326


In [16]:
pr_random_removal_nodes_uniform = calc_pagerank(G_random_removal_nodes_uniform)
df_random_removal_nodes_uniform = create_dataframe(pr_random_removal_nodes_uniform, G_random_removal_nodes_uniform)
df_random_removal_nodes_uniform.head()

rank,node,score,in edges,out edges
1,404,0.042501,9,0
2,195,0.020760,78,1
3,77,0.019746,117,2
4,728,0.017544,10,0
5,36,0.011739,167,5


## Graph evolution methodologies using statistical measures

### D. Removal of nodes at random but proportional to the degree of the nodes

Let $n$ represent the number of nodes that should be removed. If the $number\_of\_nodes$ parameter is givem then $n = number\_of\_nodes$, if this parameter is not specfied by the caller we have $n = \lfloor(0.1 * total\_number\_of\_nodes(G\_in))\rfloor)$

For $n$ iterations do the following:
* Select a node $m$ at random, but proportional to the in degree's of the nodes (iterations are abstracted by $np.random.choice$)
* Remove this node and its respective edges from the graph

In [17]:
def random_node_removals_proportional_indegree(G_in, number_given = False, number_of_nodes = None):
    if (number_given & number_of_nodes < len(list(G_in))): # check if we do not remove too much nodes
        n = number_of_nodes
    else:
        n = int(0.1 * len(list(G_in)))  # max 10% of nodes
    #remove nodes and corresponding edges
    print("number of nodes before :"+ str(len(list(G_in))))
    list_of_nodes = list(G_in)
    in_degrees = dict(G_in.in_degree()).values() # in_degrees of all the nodes
    prob_degree = [float(i)/sum(in_degrees) for i in in_degrees] # probabilities proportional to degree
    
    selected_nodes = np.random.choice(list_of_nodes, size = n, replace = False, p = prob_degree)
    for m_remove in selected_nodes:
        G_in.remove_node(m_remove)
    print("number of nodes after :"+ str(len(list(G_in))))
    return G_in

In [18]:
G_random_node_removals_proportional_indegree = random_node_removals_proportional_indegree(G.copy(), True, 100)

number of nodes before :2426
number of nodes after :2326


In [19]:
G_random_node_removals_proportional_indegree = random_node_removals_proportional_indegree(G.copy(), True, 100)
pr_random_node_removals_proportional_indegree = calc_pagerank(G_random_node_removals_proportional_indegree)
df_random_node_removals_proportional_indegree = create_dataframe(pr_random_node_removals_proportional_indegree,
                                                                 G_random_node_removals_proportional_indegree)
df_random_node_removals_proportional_indegree.head()

number of nodes before :2426
number of nodes after :2326


rank,node,score,in edges,out edges
1,404,0.035940,9,0
2,728,0.022885,10,0
3,195,0.015239,66,1
4,183,0.014663,71,3
5,182,0.011572,77,9


### E. Removal of nodes at random but proportional to the hubs/authorithy measures (HITS) of nodes

Let $n$ represent the number of nodes that should be removed. If the $number\_of\_nodes$ parameter is givem then $n = number\_of\_nodes$, if this parameter is not specfied by the caller we have $n = \lfloor(0.1 * total\_number\_of\_nodes(G\_in))\rfloor)$

For $n$ iterations do the following:
* Select a node $m$ at random, but proportional to HITS measures (i.e. hub or authority) of the nodes (iterations are abstracted by $np.random.choice$)
* Remove this node and its respective edges from the graph

In [20]:
def random_node_removals_proportional_HITS(G_in, authorithy = False, number_given = False, number_of_nodes = None):
    if (number_given & number_of_nodes < len(list(G_in))): # check if we do not remove too much nodes
        n = number_of_nodes
    else:
        n = int(0.1 * len(list(G_in)))  # max 10% of nodes
    #remove nodes and corresponding edges
    node_counter = 0
    while(node_counter < n):
        list_of_nodes = list(G_in)
        print(int((node_counter / n) * 100), "%")
        if (authorithy):
            p = list(nx.hits(G_in)[0].values()) # probabilities proportional to authority of nodes
        else: # hub
            p = list(nx.hits(G_in)[1].values()) # probabilities proportional to hub of nodes
        node_remove = np.random.choice(list_of_nodes, p = p)
        G_in.remove_node(node_remove)
        node_counter += 1
    print("number of nodes after :"+ str(len(list(G_in))))
    return G_in

In [21]:
G_random_node_removals_proportional_HITS = random_node_removals_proportional_HITS(G.copy(), True, True, 10)

0 %
10 %
20 %
30 %
40 %
50 %
60 %
70 %
80 %
90 %
number of nodes after :2416


In [22]:
pr_random_node_removals_proportional_HITS = calc_pagerank(G_random_node_removals_proportional_HITS)
df_random_node_removals_proportional_HITS = create_dataframe(pr_random_node_removals_proportional_HITS,
                                                             G_random_node_removals_proportional_HITS)
df_random_node_removals_proportional_HITS.head()

rank,node,score,in edges,out edges
1,404,0.042666,10,0
2,195,0.019819,75,1
3,77,0.018628,119,2
4,728,0.015523,10,0
5,36,0.011184,164,5


### F. Addition of nodes at random but proportional to the degree of the nodes

In [23]:
#Edge Copying Model (slide 53 of Week6-SNA-Props)
def random_node_additions_proportional_in_degree(G_in, number_given = False, number_of_nodes = 1, k = 5):
    if (number_given & number_of_nodes < len(list(G_in))): # check if we do not remove too much nodes
        n = number_of_nodes
    else:
        n = int(0.1 * len(list(G_in)))  # max 10% of nodes
    print("number of edges before :"+ str(len(G_in.edges())))
    for _ in range(n):
        #k is number of edges to be added, random integer 1 between 5
        k = random.randint(1, k) #select k random vertices
        list_of_nodes = list(G_in)  #create list of nodes
        in_degrees = dict(G_in.in_degree()).values() # in_degrees of all the nodes
        prob_degree = [float(i)/sum(in_degrees) for i in in_degrees] # probabilities proportional to degree
        k_random_selected_nodes = np.random.choice(list_of_nodes, size = k, p = prob_degree, replace = False) # selecte k nodes proportional to chosen measure
        
        new_node = nx.number_of_nodes(G_in) + 1 #add node to graph
        G_in.add_node(str(new_node))
        
        for node in k_random_selected_nodes:
            successors = list(G_in.successors(str(node)))
            for node_to in successors:
                G_in.add_edge(new_node, node_to) # add outgoing edges
            predecessors = list(G_in.predecessors(str(node)))
            for node_from in predecessors:
                G_in.add_edge(node_from, new_node) # add incoming edges
        
        # pick one node that has a low probability (relatively low number of incoming edges)
        non_zero_probs = [i for i in prob_degree if i != 0.0]
        highest_chance_nodes = np.random.choice(list_of_nodes, p = prob_degree, 
                                                size = (len(non_zero_probs) - 1), replace = False)
        
        node_to_add = random.sample(set(list_of_nodes).difference(set(highest_chance_nodes)), 1)[0] # low prob node
        successors = list(G_in.successors(node_to_add)) # successors of the node
        predecessors = list(G_in.predecessors(node_to_add)) # predecessors of the node 
        
        succ_current_node = list(G_in.successors(new_node)) # find the successors of the new node 
        pred_current_node = list(G_in.predecessors(new_node)) # find the predecessors of the new node
                                 
        # remove nodes to which the new node is already connected from the successors/predecessors list
        successors = [n for n in successors if not n in succ_current_node]
        predecessors = [n for n in predecessors if not n in pred_current_node]
                                 
        for node_to in successors:
            G_in.add_edge(new_node, node_to) # add outgoing edges
        for node_from in predecessors:
            G_in.add_edge(node_from, new_node) # add incoming edges
            
    print("number of edges after :"+str(len(G_in.edges())))
    return G_in

In [24]:
G_random_node_additions_proportional_in_degree = random_node_additions_proportional_in_degree(G.copy(), True, 10, 5)

number of edges before :16631
number of edges after :17149


In [25]:
pr_random_node_additions_proportional_in_degree = calc_pagerank(G_random_node_additions_proportional_in_degree)
df_random_node_additions_proportional_in_degree = create_dataframe(pr_random_node_additions_proportional_in_degree,
                                                             G_random_node_additions_proportional_in_degree)
df_random_node_additions_proportional_in_degree.head()

rank,node,score,in edges,out edges
1,404,0.042425,10,0
2,195,0.019814,82,1
3,77,0.018502,123,2
4,728,0.015456,10,0
5,36,0.011004,170,5


### G. Addition of nodes at random but proportional to the hubs/authorithy measures (HITS) of nodes

In [26]:
#Edge Copying Model (slide 53 of Week6-SNA-Props)
def random_node_additions_proportional_HITS(G_in, authority = False, number_given = False, number_of_nodes = 1, k = 5):
    if (number_given & number_of_nodes < len(list(G_in))): # check if we do not remove too much nodes
        n = number_of_nodes
    else:
        n = int(0.1 * len(list(G_in)))  # max 10% of nodes
    print("number of edges before :"+ str(len(G_in.edges())))
    for _ in range(n):
        #k is number of edges to be added, random integer 1 between 5
        k = random.randint(1, k) #select k random vertices
        list_of_nodes = list(G_in)  #create list of nodes
        if (authority):
            p = list(nx.hits(G_in)[0].values())
        else: # hub
            p = list(nx.hits(G_in)[1].values())
        
        k_random_selected_nodes = np.random.choice(list_of_nodes, size = k, p = p, replace = False) # selecte k nodes proportional to chosen measure
        
        new_node = nx.number_of_nodes(G_in) + 1 #add node to graph
        G_in.add_node(str(new_node))
        
        for node in k_random_selected_nodes:
            successors = list(G_in.successors(node))
            for node_to in successors:
                G_in.add_edge(new_node, node_to) # add outgoing edges
            predecessors = list(G_in.predecessors(node))
            for node_from in predecessors:
                G_in.add_edge(node_from, new_node) # add incoming edges
        
        # pick one node that has a low probability (relatively low number of incoming edges)
        non_zero_probs = [i for i in p if i != 0.0]
        highest_chance_nodes = np.random.choice(list_of_nodes, p = p, 
                                                size = (len(non_zero_probs) - 1), replace = False)
        
        node_to_add = random.sample(set(list_of_nodes).difference(set(highest_chance_nodes)), 1)[0] # low prob node
        successors = list(G_in.successors(node_to_add)) # successors of the node
        predecessors = list(G_in.predecessors(node_to_add)) # predecessors of the node 
        
        succ_current_node = list(G_in.successors(new_node)) # find the successors of the new node 
        pred_current_node = list(G_in.predecessors(new_node)) # find the predecessors of the new node
                                 
        # remove nodes to which the new node is already connected from the successors/predecessors list
        successors = [n for n in successors if not n in succ_current_node]
        predecessors = [n for n in predecessors if not n in pred_current_node]
                                 
        for node_to in successors:
            G_in.add_edge(new_node, node_to) # add outgoing edges
        for node_from in predecessors:
            G_in.add_edge(node_from, new_node) # add incoming edges
            
    print("number of edges after :"+str(len(G_in.edges())))
    return G_in

In [27]:
G_random_node_additions_proportional_HITS = random_node_additions_proportional_HITS(G.copy(), True, True, 20, 5)

number of edges before :16631
number of edges after :17899


In [28]:
pr_random_node_additions_proportional_HITS = calc_pagerank(G_random_node_additions_proportional_HITS)
df_random_node_additions_proportional_HITS = create_dataframe(pr_random_node_additions_proportional_HITS, G_random_node_additions_proportional_HITS)
df_random_node_additions_proportional_HITS.head()

rank,node,score,in edges,out edges
1,404,0.041391,10,0
2,195,0.020001,81,1
3,77,0.018687,125,2
4,728,0.014778,10,0
5,36,0.011433,177,5
